In [37]:
from dotenv import load_dotenv
load_dotenv()  # Loads from local .env by default

import os
groq_key = os.getenv("GROQ_API_KEY")

from groq import Groq
groq_client = Groq(api_key=groq_key)

print("Groq client initialized ✅")

Groq client initialized ✅


In [38]:
from langsmith import traceable

inputs = [
  {"role": "system", "content": "You are a helpful assistant. 🦜"},
  {"role": "user", "content": "I'd like to book a table for two."},
]

@traceable(
  run_type="llm",
  metadata={"ls_provider": "groq", "ls_model_name": "llama-3.1-8b-instant"}
)
def chat_model(messages: list):
  # Defensive: ensure messages is a list of dicts with 'role' and 'content'
  if not isinstance(messages, list) or not all(isinstance(m, dict) and 'role' in m and 'content' in m for m in messages):
      raise ValueError("messages must be a list of dicts with 'role' and 'content' keys")
  user_message = next((m["content"] for m in messages if m["role"] == "user"), "")
  if "table" in user_message.lower():
      suggestion = "How about The Parrot Bistro? 🦜🍽️"
      # Defensive: avoid mutating input list
      new_messages = messages + [{"role": "assistant", "content": suggestion}]
      return groq_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=new_messages,
        temperature=0.3,
      )
  return groq_client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages,
    temperature=0.3,
  )

try:
    response = chat_model(inputs)
    print(response)
except Exception as e:
    print(f"Error: {e}")

ChatCompletion(id='chatcmpl-44929bca-c5bd-4a25-ab0c-71f12e0be787', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=" It's a lovely spot with a cozy atmosphere and delicious food. Would you like to book a table for two there?", role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=None))], created=1759248105, model='llama-3.1-8b-instant', object='chat.completion', system_fingerprint='fp_90c2e79dab', usage=CompletionUsage(completion_tokens=25, prompt_tokens=69, total_tokens=94, completion_time=0.039076776, prompt_time=0.004290237, queue_time=0.044696463, total_time=0.043367013), usage_breakdown=None, x_groq={'id': 'req_01k6dnqfjjeavswcwgcm6zyrrp'}, service_tier='on_demand')


In [39]:
def _reduce_chunks(chunks: list):
    all_text = "".join([chunk["choices"][0]["message"]["content"] for chunk in chunks])
    return {"choices": [{"message": {"content": all_text, "role": "assistant"}}]}

@traceable(
    run_type="llm",
    metadata={"ls_provider": "groq", "ls_model_name": "llama-3.1-8b-instant"},
    # TODO: Add a reduce_fn
)
def my_streaming_chat_model(messages: list):
    # Stream a multi-part parrot joke
    joke_parts = [
        "Why did the parrot go to the restaurant? ",
        "Because it wanted a cracker with its dinner! 🦜🥨"
    ]
    for chunk in joke_parts:
        yield {
            "choices": [
                {
                    "message": {
                        "content": chunk,
                        "role": "assistant",
                    }
                }
            ]
        }

list(
    my_streaming_chat_model(
        [
            {"role": "system", "content": "You are a helpful assistant. Please greet the user."},
            {"role": "user", "content": "polly the parrot"},
        ],
    )
)

[{'choices': [{'message': {'content': 'Why did the parrot go to the restaurant? ',
     'role': 'assistant'}}]},
 {'choices': [{'message': {'content': 'Because it wanted a cracker with its dinner! 🦜🥨',
     'role': 'assistant'}}]}]

In [40]:
from langsmith import traceable

def _convert_docs(results):
  # Add a fun parrot fact to each document's metadata
  fun_facts = [
      "Parrots can mimic human speech! 🦜",
      "Some parrots live over 60 years.",
      "Parrots are highly intelligent birds."
  ]
  return [
      {
          "page_content": r,
          "type": "Document",
          "metadata": {"foo": "bar", "fun_fact": fun_facts[i % len(fun_facts)]}
      }
      for i, r in enumerate(results)
  ]

@traceable(
    run_type="retriever"
)
def retrieve_docs(query):
  # Retriever returning hardcoded dummy documents.
  # In production, this could be a real vector datatabase or other document index.
  contents = ["Document contents 1", "Document contents 2", "Document contents 3"]
  return _convert_docs(contents)

retrieve_docs("User query")

[{'page_content': 'Document contents 1',
  'type': 'Document',
  'metadata': {'foo': 'bar', 'fun_fact': 'Parrots can mimic human speech! 🦜'}},
 {'page_content': 'Document contents 2',
  'type': 'Document',
  'metadata': {'foo': 'bar', 'fun_fact': 'Some parrots live over 60 years.'}},
 {'page_content': 'Document contents 3',
  'type': 'Document',
  'metadata': {'foo': 'bar',
   'fun_fact': 'Parrots are highly intelligent birds.'}}]

In [41]:
from langsmith import traceable
from groq import Groq
from typing import List, Optional
import json

groq_key = os.getenv("GROQ_API_KEY")
groq_client = Groq(api_key=groq_key)

@traceable(
  # TODO: Add an run_type="tool"
)
def get_current_temperature(location: str, unit: str):
    return 65 if unit == "Fahrenheit" else 17

@traceable(run_type="llm")
def call_groq(
    messages: List[dict], tools: Optional[List[dict]]
) -> str:
  return groq_client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages,
    temperature=0,
    tools=tools
  )

@traceable(run_type="chain")
def ask_about_the_weather(inputs, tools):
  response = call_groq(inputs, tools)
  tool_call_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
  location = tool_call_args["location"]
  unit = tool_call_args["unit"]
  tool_response_message = {
    "role": "tool",
    "content": json.dumps({
        "location": location,
        "unit": unit,
        "temperature": get_current_temperature(location, unit),
    }),
    "tool_call_id": response.choices[0].message.tool_calls[0].id
  }
  inputs.append(response.choices[0].message)
  inputs.append(tool_response_message)
  output = call_groq(inputs, None)
  return output

tools = [
    {
      "type": "function",
      "function": {
        "name": "get_current_temperature",
        "description": "Get the current temperature for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": ["Celsius", "Fahrenheit"],
              "description": "The temperature unit to use. Infer this from the user's location."
            }
          },
          "required": ["location", "unit"]
        }
      }
    }
]
inputs = [
  {"role": "system", "content": "You are a helpful assistant."},
  {"role": "user", "content": "What is the weather today in New York City?"},
]

ask_about_the_weather(inputs, tools)

ChatCompletion(id='chatcmpl-de628041-954a-46be-a5df-c5ce263a303b', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Unfortunately, I'm a large language model, I don't have real-time access to current weather conditions. The information I provided was just a mock output.\n\nTo get the current weather in New York City, I suggest checking a reliable weather website or app, such as AccuWeather, Weather.com, or the National Weather Service (NWS).\n\nHowever, if you want to know the typical weather conditions for a specific time of year in New York City, I'd be happy to help with that.", role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=None))], created=1759248106, model='llama-3.1-8b-instant', object='chat.completion', system_fingerprint='fp_50a6be1b6f', usage=CompletionUsage(completion_tokens=101, prompt_tokens=107, total_tokens=208, completion_time=0.160485031, prompt_time=0.005847562, queue_time=0.044

In [42]:
import json

@traceable(run_type="chain")
def ask_about_the_weather(inputs, tools):
    response = call_groq(inputs, tools)

    # Check if tool_calls exist in the response
    tool_calls = getattr(response.choices[0].message, "tool_calls", None)
    if not tool_calls:
        print("[DEBUG] No tool calls returned by model. Full response:")
        print(response)
        return {"error": "No tool calls returned by model", "raw_response": response}

    # Extract arguments safely
    tool_call_args = json.loads(tool_calls[0].function.arguments)
    location = tool_call_args.get("location", "Unknown")
    unit = tool_call_args.get("unit", "Unknown")

    return {"location": location, "unit": unit, "raw_response": response}
